Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

# Document Classification (Assignment)

This notebook is for you to practice skills during lecture.

Today's guided module project and assignment will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a kaggle competition. We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills.

## Sections
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy
* <a href="#p4">Part 4</a>: Post Lecture Assignment

# Text Feature Extraction & Classification Pipelines (Learn)
<a id="p1"></a>

## Follow Along 

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model (try using the pipe method I just demoed)

### Load Competition Data

In [269]:
import pandas as pd

# You may need to change the path
train = pd.read_csv('./whiskey-reviews-dspt9/train.csv')
test = pd.read_csv('./whiskey-reviews-dspt9/test.csv')

In [51]:
train.head()

,id,description,ratingCategory
0,1321,"\nSometimes, when whisky is batched, a few lef...",1
1,3861,\nAn uncommon exclusive bottling of a 6 year o...,0
2,655,\nThis release is a port version of Amrut’s In...,1
3,555,\nThis 41 year old single cask was aged in a s...,1
4,1965,"\nQuite herbal on the nose, with aromas of dri...",1


In [52]:
train.description[0]

'\nSometimes, when whisky is batched, a few leftover barrels are returned to the warehouse. Canadian Club recently pulled and vatted several of these from the 1970s. Acetone, Granny Smith apples, and fresh-cut white cedar showcase this long age. Complex and spicy, yet reserved, this dram is ripe with strawberries, canned pears, cloves, pepper, and faint flowers, then slightly pulling oak tannins. Distinct, elegant, and remarkably vibrant, this ancient Canadian Club is anything but tired. (Australia only)\xa0A$133'

In [53]:
train.ratingCategory[0]

1

### Define Pipeline Components

In [54]:
# Import Statements
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_20newsgroups
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
import numpy as np
import pandas as pd

In [55]:
vect = TfidfVectorizer(stop_words='english', 
                       ngram_range=(1,2),
                       min_df=2,
                       max_df=0.25)
svm = LinearSVC()

pipe = Pipeline([('vect', vect), ('svm', svm)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [56]:
# parameters = {
#     'vect__max_df': (0.75, 1.0),
#     'clf__max_depth':(5,10,15,20)
# }

# grid_search = GridSearchCV(pipe, parameters, cv=5, n_jobs=4, verbose=1)
# grid_search.fit(train.description, train.ratingCategory)

parameters = {
    'vect__max_df': (0.3, 0.5),
    'vect__min_df': (2, 5, 10),
    'vect__max_features': (5000, 20000),
    'svm__penalty': ('l1','l2'),
    'svm__C': (0.1, 0.5, 1., 2.)
}

grid_search = GridSearchCV(pipe, parameters, cv=2, n_jobs=-1, verbose=1)
grid_search.fit(train.description, train.ratingCategory)

Fitting 2 folds for each of 96 candidates, totalling 192 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   17.7s
[Parallel(n_jobs=-1)]: Done 192 out of 192 | elapsed:   59.4s finished


GridSearchCV(cv=2, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=0.25,
                                                        max_features=None,
                                                        min_df=2,
                                                        ngram_range=(1, 2),
                                                        n

In [57]:
grid_search.best_score_

0.748471464300322

### Make a Submission File
*Note:* In a typical Kaggle competition, you are only allowed two submissions a day, so you only submit if you feel you cannot achieve higher test accuracy. For this competition the max daily submissions are capped at **20**. Submit for each demo and for your assignment. 

In [58]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [59]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [60]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [61]:
subNumber = 0

In [62]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./whiskey-reviews-dspt9/sample_submission.csv', index=False)
subNumber += 1

## Challenge

You're trying to achieve a minimum of 80% Accuracy on your model.

## Latent Semantic Indexing (Learn)
<a id="p2"></a>

## Follow Along
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
4. Make a submission to Kaggle 


In [63]:
# Apply to your Dataset

from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier

from scipy.stats import randint

param_dist = {
    
    'max_depth' : randint(3,10),
    'min_samples_leaf': randint(2,15)
}

In [64]:
# Continue Word Embedding Work Here
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer

svd = TruncatedSVD(n_components=2, algorithm='randomized', n_iter=10)

# vec = CountVectorizer()
# dtm = vec.fit_transform(data)
# dtm

In [65]:
# build a pipeline, incorporate SVD, and run a gridsearch 
# Define the Pipeline
vect = TfidfVectorizer(stop_words='english', 
                       ngram_range=(1,2),
                       min_df=2,
                       max_df=0.3)
svm = LinearSVC(C=0.5, 
                penalty='l2')

pipe = Pipeline([
    ('vect', vect),      # TF-IDF Vectorizer
    ('svd', svd),        # LSI (SVD Dim Reduction)
    ('clf', svm)         # LinearSVC Classifier
])
pipe

Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=0.3, max_features=None,
                                 min_df=2, ngram_range=(1, 2), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words='english', strip_accents=None,
                                 sublinear_tf=False,
                                 token_patte...
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('svd',
                 TruncatedSVD(algorithm='randomized', n_components=2, n_iter=10,
                              random_state=None, tol=0.0)),
                ('clf'

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [66]:
parameters = {
    'vect__max_features': (10000, 20000),
    'svd__n_components': (100, 500),
}

grid_search = GridSearchCV(pipe, parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(train.description, train.ratingCategory)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  1.4min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=0.3,
                                                        max_features=None,
                                                        min_df=2,
                                                        ngram_range=(1, 2),
                                                        no

In [67]:
grid_search.best_score_

0.7489637980206673

### Make a Submission File

In [68]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [69]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [70]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [71]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./whiskey-reviews-dspt9/sample_submission_2.csv', index=False)
subNumber += 1

## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets. 

# Word Embeddings with Spacy (Learn)
<a id="p3"></a>

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use those embeddings as your features for a classification model.
4. Make a submission to Kaggle 

## Follow Along

### Define Pipeline Components

In [72]:
import spacy

# build a model that is trained on word vectors 

# load in pre-trained w2v model 
nlp = spacy.load("en_core_web_md")

def get_word_vectors(docs):
    return [nlp(doc).vector for doc in docs]

In [73]:
#Get the vectors for each sentence (mean of all the word vectors)

X_train = get_word_vectors(train.description)
X_test = get_word_vectors(test.description)

In [74]:
#Instantiate the classifier

svm = LinearSVC(C=0.5, 
                penalty='l2')

In [75]:
#Fit the model

y_train = train.ratingCategory

svm.fit(X_train, y_train)
pred = svm.predict(X_test)

### Make a Submission File

In [76]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [77]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [78]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./whiskey-reviews-dspt9/sample_submission_3.csv', index=False)
subNumber += 1

# Post Lecture Assignment
<a id="p4"></a>

Your primary assignment this afternoon is to achieve a minimum of 80% accuracy on the Kaggle competition. Once you have achieved 70% accuracy, please work on the following: 

1. Research "Sentiment Analysis". Provide answers in markdown to the following questions: 
    - What is "Sentiment Analysis"? 
    - Is Document Classification different than "Sentiment Analysis"? Provide evidence for your response
    - How do create labeled sentiment data? Are those labels really sentiment?
    - What are common applications of sentiment analysis?
2. Research our why word embeddings worked better for the lecture notebook than on the whiskey competition.
    - This [text classification documentation](https://developers.google.com/machine-learning/guides/text-classification/step-2-5) from Google might be of interest
    - Neural Networks are becoming more popular for document classification. Why is that the case?

In [155]:
#Add to stop words

STOP_WORDS = nlp.Defaults.stop_words.union(['\n', '\r\n', ' ', '  ', '\xa0'])

In [276]:
#Create more columns in the dataset, to give the model more to work with

#Replace whisky with whiskey
def replace_w(text):
    replaced = text.replace('whisky', 'whiskey')
    return replaced
    
#use spaCy on description
def get_lemmas(text):

    lemmas = []
    doc = nlp(text)
        
    for token in doc:
        if ((token.text not in STOP_WORDS) and (token.is_punct == False)): #Use spacy to take out stop words
            lemmas.append(token.lemma_.lower())
    
    return lemmas

#Vectorize documents
def get_word_vectors(docs):
    return [nlp(doc).vector for doc in docs]

In [277]:
#Wrangle the training data

#Replace whisky with whiskey
train['description'] = train['description'].apply(replace_w)

#Tokenize train data text
train['spacy_tokens'] = train['description'].apply(get_lemmas)

In [278]:
train.head()

,id,description,ratingCategory,spacy_tokens
0,1321,"\nSometimes, when whiskey is batched, a few le...",1,"[sometimes, whiskey, batch, leftover, barrel, ..."
1,3861,\nAn uncommon exclusive bottling of a 6 year o...,0,"[an, uncommon, exclusive, bottling, 6, year, o..."
2,655,\nThis release is a port version of Amrut’s In...,1,"[this, release, port, version, amrut, intermed..."
3,555,\nThis 41 year old single cask was aged in a s...,1,"[this, 41, year, old, single, cask, age, sherr..."
4,1965,"\nQuite herbal on the nose, with aromas of dri...",1,"[quite, herbal, nose, aroma, dry, tarragon, pa..."


In [290]:

def string_text(text):
    text = str(text).strip('[]')
    return text

In [294]:
train['string'] = train['spacy_tokens'].apply(string_text)
train.head()

,id,description,ratingCategory,spacy_tokens,token_len,string
0,1321,"\nSometimes, when whiskey is batched, a few le...",1,"[sometimes, whiskey, batch, leftover, barrel, ...",51,"'sometimes', 'whiskey', 'batch', 'leftover', '..."
1,3861,\nAn uncommon exclusive bottling of a 6 year o...,0,"[an, uncommon, exclusive, bottling, 6, year, o...",52,"'an', 'uncommon', 'exclusive', 'bottling', '6'..."
2,655,\nThis release is a port version of Amrut’s In...,1,"[this, release, port, version, amrut, intermed...",52,"'this', 'release', 'port', 'version', 'amrut',..."
3,555,\nThis 41 year old single cask was aged in a s...,1,"[this, 41, year, old, single, cask, age, sherr...",53,"'this', '41', 'year', 'old', 'single', 'cask',..."
4,1965,"\nQuite herbal on the nose, with aromas of dri...",1,"[quite, herbal, nose, aroma, dry, tarragon, pa...",45,"'quite', 'herbal', 'nose', 'aroma', 'dry', 'ta..."


In [295]:
def get_word_vectors(docs):
    return [nlp(doc).vector for doc in docs]

In [296]:
X_train = get_word_vectors(train.string)

In [301]:
#Repeat pipeline with test dataset

test['description'] = test['description'].apply(replace_w)

test['spacy_tokens'] = test['description'].apply(get_lemmas)

word_counts = Counter()
test['spacy_tokens'].apply(lambda x: word_counts.update(x))
test['token_len'] = test['spacy_tokens'].map(len)

test['string'] = test['spacy_tokens'].apply(string_text)

train.head()

,id,description,ratingCategory,spacy_tokens,token_len,string
0,1321,"\nSometimes, when whiskey is batched, a few le...",1,"[sometimes, whiskey, batch, leftover, barrel, ...",51,"'sometimes', 'whiskey', 'batch', 'leftover', '..."
1,3861,\nAn uncommon exclusive bottling of a 6 year o...,0,"[an, uncommon, exclusive, bottling, 6, year, o...",52,"'an', 'uncommon', 'exclusive', 'bottling', '6'..."
2,655,\nThis release is a port version of Amrut’s In...,1,"[this, release, port, version, amrut, intermed...",52,"'this', 'release', 'port', 'version', 'amrut',..."
3,555,\nThis 41 year old single cask was aged in a s...,1,"[this, 41, year, old, single, cask, age, sherr...",53,"'this', '41', 'year', 'old', 'single', 'cask',..."
4,1965,"\nQuite herbal on the nose, with aromas of dri...",1,"[quite, herbal, nose, aroma, dry, tarragon, pa...",45,"'quite', 'herbal', 'nose', 'aroma', 'dry', 'ta..."


In [302]:
X_test = get_word_vectors(test.string)

In [303]:
#Instantiate the classifier

svm = LinearSVC(C=0.5)

In [304]:
#Create the X matrices and y target

y_train = train.ratingCategory

In [305]:
#Fit the model
svm.fit(X_train, y_train)
pred = svm.predict(X_test)

In [306]:
#Make a submission file

submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [307]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./whiskey-reviews-dspt9/sample_submission_4.csv', index=False)

### Second Attempt

In [246]:
#Using LSI model on cleaned data

svd = TruncatedSVD(n_components=2, algorithm='randomized', n_iter=10)

vect = TfidfVectorizer(stop_words='english', 
                       ngram_range=(1,2),
                       min_df=2,
                       max_df=0.3)

svm = LinearSVC(C=0.5, penalty='l2')

pipe = Pipeline([
    ('vect', vect),      # TF-IDF Vectorizer
    ('svd', svd),        # LSI (SVD Dim Reduction)
    ('clf', svm)         # LinearSVC Classifier
])

In [247]:
parameters = {
    'vect__max_df': (0.3, 0.5),
    'vect__min_df': (5, 10),
    'vect__max_features': (10000, 20000),
    'clf__penalty': ('l1','l2'),
    'clf__C': (0.5, 1)
}

grid_search = GridSearchCV(pipe, parameters, cv=3, n_jobs=-1, verbose=1)
grid_search.fit(train.description, train.ratingCategory)

Fitting 3 folds for each of 32 candidates, totalling 96 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   19.3s
[Parallel(n_jobs=-1)]: Done  96 out of  96 | elapsed:   37.2s finished


GridSearchCV(cv=3, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=0.3,
                                                        max_features=None,
                                                        min_df=2,
                                                        ngram_range=(1, 2),
                                                        no

In [249]:
grid_search.best_score_

0.7058967704262967